# Cleaning US Census Data

You just got hired as a Data Analyst at the Census Bureau, which collects census data and creates interesting visualizations and insights from it.

The person who had your job before you left you all the data they had for the most recent census. It is in multiple `csv` files. They didn't use pandas, they would just look through these `csv` files manually whenever they wanted to find something. Sometimes they would copy and paste certain numbers into Excel to make charts.

The thought of it makes you shiver. This is not scalable or repeatable.

Your boss wants you to make some scatterplots and histograms by the end of the day. Can you get this data into `pandas` and into reasonable shape so that you can make these histograms?

## Inspect the Data!

1. The first visualization your boss wants you to make is a scatterplot that shows average income in a state vs proportion of women in that state.

   Open some of the census `csv` files that came with the kit you downloaded. How are they named? What kind of information do they hold? Will they help us make this graph?

In [ ]:
import csv

In [ ]:
with open('states1.csv') as csv1:
    print(csv1.read())

2. It will be easier to inspect this data once we have it in a DataFrame. You can't even call `.head()` on these `csv`s! How are you supposed to read them?

   Using `glob`, loop through the census files available and load them into DataFrames. Then, concatenate all of those DataFrames together into one DataFrame, called something like `us_census`.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import re
import seaborn as sns

3. Look at the `.columns` and the `.dtypes` of the `us_census` DataFrame. Are those datatypes going to hinder you as you try to make histograms?

In [ ]:
files = glob.glob('states*.csv')

In [ ]:
df_list = [pd.read_csv(filename) for filename in files]

In [ ]:
us_census = pd.concat(df_list)

In [ ]:
us_census.columns

In [ ]:
us_census.dtypes

In [ ]:
us_census.head(10)

4. Look at the `head()` of the DataFrame so that you can understand why some of these `dtypes` are objects instead of integers or floats.

   Start to make a plan for how to convert these columns into the right types for manipulation.

## Regex to the Rescue

5. Use regex to turn the `Income` column into a format that is ready for conversion into a numerical type.

In [ ]:
us_census.Income = pd.to_numeric(us_census['Income'].replace('[\$,]', '', regex=True))

In [ ]:
us_census.head()

In [ ]:
us_census.shape

In [ ]:
us_census.Income.dtypes

6. Look at the `GenderPop` column. We are going to want to separate this into two columns, the `Men` column, and the `Women` column.

   Split the column into those two new columns using `str.split` and separating out those results.

In [ ]:
GenderPop_split = us_census.GenderPop.str.split('_')

In [ ]:
us_census['Men'] = GenderPop_split.str.get(0)
us_census['Women'] = GenderPop_split.str.get(1)

7. Convert both of the columns into numerical datatypes.

   There is still an `M` or an `F` character in each entry! We should remove those before we convert.

In [ ]:
us_census.Men.dtypes

In [ ]:
us_census.Men = pd.to_numeric(us_census.Men.str[:-1])
us_census.Women = pd.to_numeric(us_census.Women.str[:-1])

In [ ]:
us_census.Women = us_census.Women

In [ ]:
us_census.Men.dtypes

In [ ]:
us_census.Men.head()

8. Now you should have the columns you need to make the graph and make sure your boss does not slam a ruler angrily on your desk because you've wasted your whole day cleaning your data with no results to show!

   Use matplotlib to make a scatterplot!
   
   ```py
   plt.scatter(the_women_column, the_income_column)
   ```
   
   Remember to call `plt.show()` to see the graph!

In [ ]:
plt.figure(figsize=[6, 6])
sns.scatterplot(x='Women', y='Income', data=us_census)
plt.title('Scatter plot of Women and Income per states', fontsize=11)
plt.xlabel('Number of women (10 million)', fontsize=9)
plt.ylabel('Average income (US $)')
plt.show()
plt.clf()

9. You want to double check your work. You know from experience that these monstrous csv files probably have `nan` values in them! Print out your column with the number of women per state to see.

   We can fill in those `nan`s by using pandas' `.fillna()` function.
   
   You have the `TotalPop` per state, and you have the `Men` per state. As an estimate for the `nan` values in the `Women` column, you could use the `TotalPop` of that state minus the `Men` for that state.
   
   Print out the `Women` column after filling the `nan` values to see if it worked!

In [ ]:
us_census.isna().sum()

In [ ]:
us_census.Women = us_census.Women.fillna(us_census.TotalPop - us_census.Men)

In [ ]:
us_census.isna().sum()

10. We forgot to check for duplicates! Use `.duplicated()` on your `census` DataFrame to see if we have duplicate rows in there.

In [ ]:
us_census.duplicated(subset='State').value_counts()

11. Drop those duplicates using the `.drop_duplicates()` function.

In [ ]:
us_census = us_census.drop_duplicates(subset='State')

In [ ]:
us_census.duplicated(subset='State').value_counts()

12. Make the scatterplot again. Now, it should be perfect! Your job is secure, for now.

In [ ]:
us_census.Women.head()

In [ ]:
plt.figure(figsize=[6, 6])
sns.scatterplot(x='Women', y='Income', data=us_census, color='orange')
plt.title('Scatter plot of Women and Income in all states', fontsize=11)
plt.xlabel('Number of women (10 million)', fontsize=9)
plt.ylabel('Average income (US $)')
plt.show()
plt.clf()

## Histogram of Races

13. Now your boss wants you to make a bunch of histograms out of the race data that you have. Look at the `.columns` again to see what the race categories are.

In [ ]:
us_census.columns

In [ ]:
us_census[['Hispanic', 'White', 'Black',
       'Native', 'Asian', 'Pacific']].head(6)

In [ ]:
us_census[['Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific']] = us_census[['Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific']].replace('[%]', '', regex=True)

In [ ]:
us_census.isna().sum()

In [ ]:
us_census.Hispanic = pd.to_numeric(us_census.Hispanic)
us_census.White = pd.to_numeric(us_census.White)
us_census.Black = pd.to_numeric(us_census.Black)
us_census.Native = pd.to_numeric(us_census.Native)
us_census.Asian = pd.to_numeric(us_census.Asian)

In [ ]:
us_census.Pacific = us_census.Pacific.fillna(1 - (us_census.Hispanic + us_census.White + us_census.Black + us_census.Native + us_census.Asian))

In [ ]:
us_census[['Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific']].dtypes

In [ ]:
us_census.Pacific = pd.to_numeric(us_census.Pacific)

In [ ]:
us_census[['Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific']].dtypes

14. Try to make a histogram for each one!

    You will have to get the columns into the numerical format, and those percentage signs will have to go.
    
    Don't forget to fill the `nan` values with something that makes sense! You probably dropped the duplicate rows when making your last graph, but it couldn't hurt to check for duplicates again.

In [ ]:
races = ['Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific']

In [ ]:
for race in races:
    plt.figure(figsize=[6,6])
    sns.displot(us_census[race])
    plt.title(f'Histogram of the proportion of {race} population', fontsize=11)
    plt.xlabel('Proportion (range)', fontsize=9)
    plt.ylabel('Frequency (number of states)', fontsize=9)
    plt.show()
    plt.clf()

## Get Creative

15. Phew. You've definitely impressed your boss on your first day of work.

    But is there a way you really convey the power of pandas and Python over the drudgery of `csv` and Excel?
    
    Try to make some more interesting graphs to show your boss, and the world! You may need to clean the data even more to do it, or the cleaning you have already done may give you the ease of manipulation you've been searching for.